## Классификация текстов: Spam or Ham

В этом задании на примере классического датасета Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) мы попробуем сделать свой спам-фильтр c помощью библиотеки scikit-learn. Датасет содержит корпус текстов 5,574 смс с метками "spam" и "ham". 

### Данные

Для удобства данные приложены в описании задания

In [260]:
import pandas as pd
df = pd.read_csv('3_data.csv', encoding='latin-1')

In [261]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


Оставляем только интересующие нас колонки — тексты смс и метки:

In [262]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Удаляем дублирующиеся тексты:

In [263]:
df = df.drop_duplicates('text')

Заменяем метки на бинарные:

In [264]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Предобработка текста (Задание 1)

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [265]:
df['text'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [266]:
from nltk import stem
from nltk.corpus import stopwords
import re
import string 

stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

text = re.sub(r'[^\w\s]', '', df['text'][0])
text = text.lower()

text_tokens = text.split()
text_tokens
text_list=[]
for word in text_tokens:
    if word not in stopwords and word not in string.punctuation:
        text_list.append(word)
text_list_stem = []
for word in text_list:
    text_list_stem.append(stemmer.stem(word))
print(' '.join(text_list_stem))

go jurong point crazi avail bugi n great world la e buffet cine got amor wat


In [267]:
from nltk import stem
from nltk.corpus import stopwords
import re
import string  

stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    text_tokens = text.split()
    text_list=[]
    for word in text_tokens:
        if word not in stopwords and word not in string.punctuation:
            text_list.append(word)
    text_list_stem = []
    for word in text_list:
        text_list_stem.append(stemmer.stem(word))
    return ' '.join(text_list_stem)


Проверка, что функция работает верно

In [268]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [269]:
df['text'] = df['text'].apply(preprocess)

In [270]:
df

,label,text
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri 2 wkli comp win fa cup final tkts 2...
3,0,u dun say earli hor u c alreadi say
4,0,nah dont think goe usf live around though
...,...,...
5567,1,2nd time tri 2 contact u u å750 pound prize 2 ...
5568,0,ì_ b go esplanad fr home
5569,0,piti mood soani suggest
5570,0,guy bitch act like id interest buy someth els ...


### Разделение данных на обучающую и тестовую выборки (Задание 2)

In [271]:
y = df['label']
X = df['text']

In [272]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: label, Length: 5169, dtype: int64

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [273]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [274]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [275]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#обучаем подель SVM

model = LinearSVC(random_state = 41, C = 1.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

/Users/muzafarov/anaconda3/envs/ML_OE/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Для самопроверки. Если вы верно дополнили функцию ```preprocess```, то должны получиться следующие результаты оценки модели.

In [276]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.977     0.994     0.986       898
           1      0.958     0.846     0.898       136

    accuracy                          0.975      1034
   macro avg      0.968     0.920     0.942      1034
weighted avg      0.975     0.975     0.974      1034



Выполним предсказание для конкретного текста

In [277]:
txt = "Take your prize, more than 100 computers, smartphones and TVs are supposed to be played in a free quiz. Call by phone 8 800 243 456"
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [278]:
model.predict(txt)

array([1])

Сообщение помечено как спам.